<a href="https://colab.research.google.com/github/mlabonne/llm-course/blob/main/Fine_tune_Llama_2_in_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2 in Google Colab
> 🗣️ Large Language Model Course

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne), based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da). Special thanks to Tolga HOŞGÖR for his solution to empty the VRAM.

This notebook runs on a T4 GPU. (Last update: 01 Aug 2023)


In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-gptq 0.7.1 requires accelerate>=0.26.0, but you have accelerate 0.21.0 which is incompatible.
auto-gptq 0.7.1 requires peft>=0.5.0, but you have peft 0.4.0 which is incompatible.


In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [1]:
from datasets import DatasetDict, load_dataset

# full_dataset = load_dataset("grammarly/coedit")
# print(full_dataset)

# train_dataset = load_dataset("grammarly/coedit", split="train[:50000]")
# test_dataset = load_dataset("grammarly/coedit", split="train[10000:]")
# # test_dataset = load_dataset("grammarly/coedit", split="validation")

all_dataset = load_dataset("grammarly/coedit", split="train+validation")
# print(all_dataset)

# print()
# print(f"train set {set(all_dataset['task'])}")
# print(f"total len: {len(all_dataset)}")
# print(f"gec len: {len(all_dataset.filter(lambda x: x['task'] == 'gec'))}")
# print(f"simplification len: {len(all_dataset.filter(lambda x: x['task'] == 'simplification'))}")
# print(f"clarity len: {len(all_dataset.filter(lambda x: x['task'] == 'clarity'))}")
# print(f"coherence len: {len(all_dataset.filter(lambda x: x['task'] == 'coherence'))}")
# print(f"paraphrase len: {len(all_dataset.filter(lambda x: x['task'] == 'paraphrase'))}")
# print(f"neutralize len: {len(all_dataset.filter(lambda x: x['task'] == 'neutralize'))}")
# print()

# train_ratio = 0.01
# test_ratio = 0.001
# train_ratio = 0.1
# test_ratio = 0.01
train_ratio = 0.02
test_ratio = 0.002
# train_ratio = 0.9
# test_ratio = 0.1

gec_dataset = all_dataset.filter(lambda x: x["task"] == "gec")
train_gec_dataset = gec_dataset.select(range(0, int(train_ratio * len(gec_dataset))))
test_gec_dataset = gec_dataset.select(range(int((1 - test_ratio) * len(gec_dataset)), len(gec_dataset)))

simplification_dataset = all_dataset.filter(lambda x: x["task"] == "simplification")
train_simplification_dataset = simplification_dataset.select(range(0, int(train_ratio * len(simplification_dataset))))
test_simplification_dataset = simplification_dataset.select(
    range(int((1 - test_ratio) * len(simplification_dataset)), len(simplification_dataset))
)

clarity_dataset = all_dataset.filter(lambda x: x["task"] == "clarity")
train_clarity_dataset = clarity_dataset.select(range(0, int(train_ratio * len(clarity_dataset))))
test_clarity_dataset = clarity_dataset.select(range(int((1 - test_ratio) * len(clarity_dataset)), len(clarity_dataset)))

coherence_dataset = all_dataset.filter(lambda x: x["task"] == "coherence")
train_coherence_dataset = coherence_dataset.select(range(0, int(train_ratio * len(coherence_dataset))))
test_coherence_dataset = coherence_dataset.select(
    range(int((1 - test_ratio) * len(coherence_dataset)), len(coherence_dataset))
)

paraphrase_dataset = all_dataset.filter(lambda x: x["task"] == "paraphrase")
train_paraphrase_dataset = paraphrase_dataset.select(range(0, int(train_ratio * len(paraphrase_dataset))))
test_paraphrase_dataset = paraphrase_dataset.select(
    range(int((1 - test_ratio) * len(paraphrase_dataset)), len(paraphrase_dataset))
)

neutralize_dataset = all_dataset.filter(lambda x: x["task"] == "neutralize")
neutralize_dataset_split = int(train_ratio * len(neutralize_dataset))
train_neutralize_dataset = neutralize_dataset.select(range(0, int(train_ratio * len(neutralize_dataset))))
test_neutralize_dataset = neutralize_dataset.select(
    range(int((1 - test_ratio) * len(neutralize_dataset)), len(neutralize_dataset))
)

from datasets import concatenate_datasets

train_dataset = concatenate_datasets(
    [
        train_gec_dataset,
        train_simplification_dataset,
        train_clarity_dataset,
        train_coherence_dataset,
        train_paraphrase_dataset,
        train_neutralize_dataset,
    ]
)
train_dataset = train_dataset.map(
    lambda item: {
        "input": item["src"],
        "reference": item["tgt"],
        "references": [item["tgt"]],
    },
    remove_columns=["src", "tgt", "_id"],
)
print(f"train set {set(train_dataset['task'])}")
print(train_dataset)

test_dataset = concatenate_datasets(
    [
        test_gec_dataset,
        test_simplification_dataset,
        test_clarity_dataset,
        test_coherence_dataset,
        test_paraphrase_dataset,
        test_neutralize_dataset,
    ]
)
test_dataset = test_dataset.map(
    lambda item: {
        "input": item["src"],
        "reference": item["tgt"],
        "references": [item["tgt"]],
    },
    remove_columns=["src", "tgt", "_id"],
)
print(f"test set {set(test_dataset['task'])}")
print(test_dataset)


def llama2_prompt(item, response=True):
    prompt = f"""<s> [INST] <<SYS>> You are a grammar assistant, you output a corrected sentence only without any explanation. <</SYS>> {item['input']} [/INST]
""".strip()
    if response:
        prompt = f"""{prompt} {item['reference']} </s>
""".strip()
    return prompt


def add_prompt(item):
    return {
        "request": f"{item['input']}\nResponse:",
        "prompt": f"{item['input']}\nResponse:{item['reference']}",
        "llama2_request": llama2_prompt(item, response=False),
        "llama2_prompt": llama2_prompt(item),
    }


dataset = DatasetDict({"train": train_dataset, "test": test_dataset})
# dataset = dataset.rename_column("task", "label")
dataset = dataset.map(add_prompt)
print(dataset)
print(dataset["train"][0])

train set {'clarity', 'neutralize', 'gec', 'simplification', 'coherence', 'paraphrase'}
Dataset({
    features: ['task', 'input', 'reference', 'references'],
    num_rows: 1413
})
test set {'clarity', 'neutralize', 'gec', 'simplification', 'coherence', 'paraphrase'}
Dataset({
    features: ['task', 'input', 'reference', 'references'],
    num_rows: 144
})


Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['task', 'input', 'reference', 'references', 'request', 'prompt', 'llama2_request', 'llama2_prompt'],
        num_rows: 1413
    })
    test: Dataset({
        features: ['task', 'input', 'reference', 'references', 'request', 'prompt', 'llama2_request', 'llama2_prompt'],
        num_rows: 144
    })
})
{'task': 'gec', 'input': 'Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert.', 'reference': 'For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean water to the desert.', 'references': ['For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean water to the desert.'], 'request': 'Remove all grammatical errors from

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
# new_model = "llama-2-7b-miniguanaco"
new_model = "llama-2-7b-coedit"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 100

# Batch size per GPU for evaluation
per_device_eval_batch_size = 100

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [4]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
print(model.config)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaConfig {
  "_name_or_path": "NousResearch/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": false,
  "vocab_size": 32000
}



In [6]:
# %%script echo skip

raw_input = "Tell me about yourself".strip()
print(raw_input)

single_input = """<s> [INST] <<SYS>> You are a grammar assistant, you output a corrected sentence only without any explanation. <</SYS>>
Make the sentence simple: Hugo Wolf was born in Windischgra ̈ tz in the Duchy of Styria (now Slovenj Gradec, Slovenia), then a part of the Austrian Empire. [/INST]
""".strip()
print(single_input)

batch_input = [
    """<s> [INST] <<SYS>> You are a grammar assistant, you output a corrected sentence only without any explanation. <</SYS>>
Make the sentence simple: Hugo Wolf was born in Windischgra ̈ tz in the Duchy of Styria (now Slovenj Gradec, Slovenia), then a part of the Austrian Empire. [/INST]
""".strip(),
    """<s> [INST] <<SYS>> You are a grammar assistant, you output a corrected sentence only without any explanation. <</SYS>>
Make the sentence simple: Handzus ̌ played for the St. Louis Blues, Phoenix Coyotes, Philadelphia Flyers, Los Angeles Kings, San Jose Sharks and the Chicago Blackhawks, with whom he won the Stanley Cup with in 2013. [/INST]
""".strip(),
]

# padding - https://huggingface.co/docs/transformers/en/pad_truncation
# inputs = tokenizer(single_input, padding=True, return_tensors="pt", return_attention_mask=False).to(device)
inputs = tokenizer(raw_input, padding=True, return_tensors="pt", return_attention_mask=False).to("cuda")
print(inputs)

outputs = model.generate(inputs.input_ids)
print(outputs)

result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# print(result)

final_result = outputs[:, inputs.input_ids.shape[1] :]
final_result = tokenizer.batch_decode(final_result, skip_special_tokens=True)
print(f"final_result: {final_result}")

Tell me about yourself
<s> [INST] <<SYS>> You are a grammar assistant, you output a corrected sentence only without any explanation. <</SYS>>
Make the sentence simple: Hugo Wolf was born in Windischgra ̈ tz in the Duchy of Styria (now Slovenj Gradec, Slovenia), then a part of the Austrian Empire. [/INST]
{'input_ids': tensor([[    1, 24948,   592,  1048,  7535]], device='cuda:0')}


/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    1, 24948,   592,  1048,  7535, 29892,   322,  2020,   366,   864,
           304,   367,   263,   760,   310,   445,  2060, 29973,    13, 19838]],
       device='cuda:0')
final_result: [', and why you want to be a part of this project?\n Unterscheidung']


In [7]:
print(dataset["train"])

Dataset({
    features: ['task', 'input', 'reference', 'references', 'request', 'prompt', 'llama2_request', 'llama2_prompt'],
    num_rows: 1413
})


In [27]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    dataset_text_field="llama2_prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [28]:
model = trainer.model
model.config.use_cache = False

print(type(tokenizer))
# print(tokenizer.add_eos_token)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print(tokenizer.padding_side)


max_length = 350
max_new_tokens = 100
max_batch = 1

print(dataset["train"])

batch_size = len(dataset["train"]) if len(dataset["train"]) < max_batch else max_batch
input_batch = dataset["train"].select(range(batch_size))
# print(f"task: {input_batch['task']}")
# print(f"input: {input_batch['input']}")
# print(f"request: {input_batch['request']}")
print(f"llama2_request: {input_batch['llama2_request']}")
print(f"reference: {input_batch['reference']}")

inputs = tokenizer(input_batch["llama2_request"], return_tensors="pt", padding=True).to(0)
print(inputs)

# model.config.pad_token_id = model.config.eos_token_id
# outputs = model.generate(**input_ids, max_new_tokens=128)
# outputs = model.generate(**inputs, max_new_tokens=128, num_return_sequences=1)
outputs = model.generate(
    **inputs,
    # max_length=max_length,
    max_new_tokens=max_new_tokens,
    # pad_token_id=tokenizer.eos_token_id,
    # eos_token_id=tokenizer.eos_token_id,
    num_return_sequences=1,
)
# outputs = model.generate(
#     **inputs,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     pad_token_id=tokenizer.eos_token_id,
#     # return_attention_mask=True,
#     # max_length=max_length,
#     max_new_tokens=max_new_tokens
# )
print(outputs)

final_result = outputs[:, inputs.input_ids.shape[1] :]
# result = tokenizer.batch_decode(trimmed_output, skip_special_tokens=False)
# result = tokenizer.batch_decode(outputs, skip_special_tokens=False)
result = tokenizer.batch_decode(final_result, skip_special_tokens=True)
print(f"result: {result}")

<class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>
2
2
right
Dataset({
    features: ['task', 'input', 'reference', 'references', 'request', 'prompt', 'llama2_request', 'llama2_prompt'],
    num_rows: 1413
})
llama2_request: ['<s> [INST] <<SYS>> You are a grammar assistant. You just provide the rewritten corrected sentence in your output, that is. You do not provide any explanation. You do not prefix your sentence with any of your comment. You end your response right after the corrected sentence. <</SYS>> Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert. [/INST]']
reference: ['For example, countries with a lot of deserts can transform their desert to increase their habitable land and use irrigation to provide clean water to the desert.']
{'input_ids': tensor([[    1,     1,   518, 25580, 29962,  3532,

/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


tensor([[    1,     1,   518, 25580, 29962,  3532, 14816, 29903,  6778,   887,
           526,   263, 25437, 20255, 29889,   887,   925,  3867,   278,   337,
         17625, 24114, 10541,   297,   596,  1962, 29892,   393,   338, 29889,
           887,   437,   451,  3867,   738,  8252, 29889,   887,   437,   451,
         10944,   596, 10541,   411,   738,   310,   596,  3440, 29889,   887,
          1095,   596,  2933,  1492,  1156,   278, 24114, 10541, 29889,   529,
           829, 14816, 29903,  6778, 15154,   599, 14961,  2922,   936,  4436,
           515,   445,  1426, 29901,  1152,  1342, 29892, 10916,   411,   263,
          3287,   310, 18197, 29879,   508, 15087,   689,  1009, 18197,   304,
          7910,  1009,  4760,   519,  2982,   322,   773,  3805,  8966,   362,
           304,  3867,  5941,  4094,   304,   278, 18197, 29889,   518, 29914,
         25580, 29962, 29871, 18585, 29892,  1244,   338,   278, 24114, 10541,
         29901,    13,    13,  3981,  2722,   411,  

In [24]:
# %%script echo skip

singnle_input = """"<s>[INST] <<SYS>>
You're a grammar assistant. You just rewrite incorrect sentences to make them correct.
You do not provide any explanation. You end your response right after the corrected sentence.
<</SYS>>
Write a paraphrase for the sentence: I don't think this is a dream. [/INST]"
""".strip()

batch_input = [
    """<s>[INST] <<SYS>>
You're a grammar assistant. You just rewrite incorrect sentences to make them correct.
You do not provide any explanation. You end your response right after the corrected sentence.
<</SYS>>
Make the sentence simple: Hugo Wolf was born in Windischgra ̈ tz in the Duchy of Styria (now Slovenj Gradec, Slovenia), then a part of the Austrian Empire. [/INST]
""".strip(),
    """<s>[INST] <<SYS>>
You're a grammar assistant. You just rewrite incorrect sentences to make them correct.
You do not provide any explanation. You end your response right after the corrected sentence.
<</SYS>>
Make the sentence simple: Handzus ̌ played for the St. Louis Blues, Phoenix Coyotes, Philadelphia Flyers, Los Angeles Kings, San Jose Sharks and the Chicago Blackhawks, with whom he won the Stanley Cup with in 2013. [/INST]
""".strip(),
]

# padding - https://huggingface.co/docs/transformers/en/pad_truncation
# inputs = tokenizer(input, return_tensors="pt", return_attention_mask=False).to(device)
# inputs = tokenizer(batch_input, padding=True, return_tensors="pt", return_attention_mask=False).to(device)
# inputs = tokenizer(batch_less_input, padding=True, truncation=True, max_length=100, return_tensors="pt", return_attention_mask=False).to(device)
# inputs = tokenizer(coedit_input_batch, padding=True, return_tensors="pt", return_attention_mask=False).to(device)
inputs = tokenizer(singnle_input, padding=True, return_tensors="pt", return_attention_mask=False).to(0)
# inputs = tokenizer(coedit_input, return_tensors="pt", return_attention_mask=False).to(device)
print(inputs)

# print(model)
# outputs = model.generate(**inputs)
outputs = model.generate(inputs.input_ids)
print(outputs)

result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# print(result)

final_result = outputs[:, inputs.input_ids.shape[1] :]
final_result = tokenizer.batch_decode(final_result, skip_special_tokens=True)
print(f"final_result: {final_result}")

{'input_ids': tensor([[    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          3492, 29915,   276,   263, 25437, 20255, 29889,   887,   925, 10683,
         10240, 25260,   304,  1207,   963,  1959, 29889,    13,  3492,   437,
           451,  3867,   738,  8252, 29889,   887,  1095,   596,  2933,  1492,
          1156,   278, 24114, 10541, 29889,    13, 29966,   829, 14816, 29903,
          6778,    13,  9984,   278, 10541,  2560, 29901, 20650, 10441,   471,
          6345,   297, 17311,   783,  3874, 29871, 31719,   260, 29920,   297,
           278, 23568, 29891,   310,   624, 29891,  2849,   313,  3707, 24917,
         29926,  4989,  7099, 29892, 24917,   423,   511,   769,   263,   760,
           310,   278,  3330,  6392, 13378, 29889,   518, 29914, 25580, 29962,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2],
        [    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,


In [16]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
# prompt = "What is a large language model?"
prompt = """<<SYS>> You're a grammar assistant, you provide a rewriten sentence only without any comments. <</SYS>>
Fix coherence mistakes in this sentence: Jump up ^ Colgrave & Mynors pp. xxxix -- xl. Colgrave & Mynors is bede's Ecclesiastical History.
""".strip()
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, early_stopping=True, num_return_sequences=1)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/izlobin/miniconda3/envs/pytorch-3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] <<SYS>> You're a grammar assistant, you provide a rewriten sentence only without any comments. <</SYS>>
Fix coherence mistakes in this sentence: Jump up ^ Colgrave & Mynors pp. xxxix -- xl. Colgrave & Mynors is bede's Ecclesiastical History. [/INST] Jump up ^ Colgrave & Mynors pp. xxxix -- xl. Colgrave & Mynors is bedes Ecclesiastical History.</s> [/INST] Jump up ^ Colgrave & Mynors pp. xxxix -- xl. Colgrave & Mynors is bedes Ecclesiastical History.</s> [/INST] Jump up ^ Colgrave & Mynors pp. xxxix -- xl. Colgrave & Mynors is


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

19965

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mlabonne/llama-2-7b-miniguanaco/commit/c81a32fd0b4d39e252326e639d63e75aa68c9a4a', commit_message='Upload tokenizer', commit_description='', oid='c81a32fd0b4d39e252326e639d63e75aa68c9a4a', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
# input
sequence = [    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,   887,
           526,   263, 25437, 20255, 29889,   887,   925,  3867,   278,   337,
         17625, 24114, 10541,   297,   596,  1962, 29892,   393,   338, 29889,
           887,   437,   451,  3867,   738,  8252, 29889,   887,   437,   451,
         10944,   596, 10541,   411,   738,   310,   596,  3440, 29889,   887,
          1095,   596,  2933,  1492,  1156,   278, 24114, 10541, 29889,   529,
           829, 14816, 29903,  6778, 15154,   599, 14961,  2922,   936,  4436,
           515,   445,  1426, 29901,  1152,  1342, 29892, 10916,   411,   263,
          3287,   310, 18197, 29879,   508, 15087,   689,  1009, 18197,   304,
          7910,  1009,  4760,   519,  2982,   322,   773,  3805,  8966,   362,
           304,  3867,  5941,  4094,   304,   278, 18197, 29889,   518, 29914,
         25580, 29962]
print(tokenizer.decode(sequence))
print()
# input
sequence = [    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,   887,
           526,   263, 25437, 20255, 29889,   887,   925,  3867,   278,   337,
         17625, 24114, 10541,   297,   596,  1962, 29892,   393,   338, 29889,
           887,   437,   451,  3867,   738,  8252, 29889,   887,   437,   451,
         10944,   596, 10541,   411,   738,   310,   596,  3440, 29889,   887,
          1095,   596,  2933,  1492,  1156,   278, 24114, 10541, 29889,   529,
           829, 14816, 29903,  6778, 15154,   599, 14961,  2922,   936,  4436,
           515,   445,  1426, 29901,  1152,  1342, 29892, 10916,   411,   263,
          3287,   310, 18197, 29879,   508, 15087,   689,  1009, 18197,   304,
          7910,  1009,  4760,   519,  2982,   322,   773,  3805,  8966,   362,
           304,  3867,  5941,  4094,   304,   278, 18197, 29889,   518, 29914,
         25580, 29962,  1152,  1342, 29892, 10916,   411,   263,  3287,   310,
         18197, 29879,   508, 15087,   689,  1009, 18197,   304,  7910,  1009,
          4760,   519,  2982,   322,   671,  3805,  8966,   362,   304,  3867,
          5941,  4094,   304,   278, 18197, 21106, 29879, 29958]
print(tokenizer.decode(sequence))
print()

# eos
sequence = [21106, 29879, 29958]
print(tokenizer.decode(sequence))
print()

# eos
sequence = [2]
print(tokenizer.decode(sequence))
print()

bos_token = tokenizer.encode('<s>')
eos_token = tokenizer.encode('</s>')

print('BOS token:', bos_token)
print('EOS token:', eos_token)

# sequence = [27914,   477, 14599, 44935,  8563,   422,   428,  2420,    25,  1114,
#           1672,    11,  2678,   351,   257,  1256,   286, 45288,   460,  1059,
#            430,   687,   511, 10326,   284,  2620,   511, 49055,  1956,   290,
#           1262, 35425,   284,  2148,  3424,  1660,   284,   262, 10326,    13,
#            198, 31077,    25,   220]
# print(tokenizer.decode(token_ids=sequence))
# sequence = [25, 220, 25]
# print(tokenizer.decode(sequence))
# sequence = [1114]
# print(tokenizer.decode(sequence))

<s><s>[INST] <<SYS>> You are a grammar assistant. You just provide the rewritten corrected sentence in your output, that is. You do not provide any explanation. You do not prefix your sentence with any of your comment. You end your response right after the corrected sentence. <</SYS>> Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert. [/INST]

<s><s>[INST] <<SYS>> You are a grammar assistant. You just provide the rewritten corrected sentence in your output, that is. You do not provide any explanation. You do not prefix your sentence with any of your comment. You end your response right after the corrected sentence. <</SYS>> Remove all grammatical errors from this text: For example, countries with a lot of deserts can terraform their desert to increase their habitable land and using irrigation to provide clean water to the desert.